In [1]:
# %%
import sys

import sklearn
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader

sys.path.append('..')
import torch
import numpy as np
import matplotlib.pyplot as plt

from lens.utils.relu_nn import get_reduced_model, prune_features
from lens import logic
import lens
from data import ConceptToTaskDataset
torch.manual_seed(0)
np.random.seed(0)



In [2]:
dataset = ConceptToTaskDataset("../data/CUB_200_2011", predictions=True)

X = dataset.attributes
y = np.asarray(dataset.targets)

concept_names = dataset.attribute_names
concept_names

['has_bill_shape::curved_(up_or_down)\n',
 'has_bill_shape::dagger\n',
 'has_bill_shape::hooked\n',
 'has_bill_shape::needle\n',
 'has_bill_shape::hooked_seabird\n',
 'has_bill_shape::spatulate\n',
 'has_bill_shape::all-purpose\n',
 'has_bill_shape::cone\n',
 'has_bill_shape::specialized\n',
 'has_wing_color::blue\n',
 'has_wing_color::brown\n',
 'has_wing_color::iridescent\n',
 'has_wing_color::purple\n',
 'has_wing_color::rufous\n',
 'has_wing_color::grey\n',
 'has_wing_color::yellow\n',
 'has_wing_color::olive\n',
 'has_wing_color::green\n',
 'has_wing_color::pink\n',
 'has_wing_color::orange\n',
 'has_wing_color::black\n',
 'has_wing_color::white\n',
 'has_wing_color::red\n',
 'has_wing_color::buff\n',
 'has_upperparts_color::blue\n',
 'has_upperparts_color::brown\n',
 'has_upperparts_color::iridescent\n',
 'has_upperparts_color::purple\n',
 'has_upperparts_color::rufous\n',
 'has_upperparts_color::grey\n',
 'has_upperparts_color::yellow\n',
 'has_upperparts_color::olive\n',
 'has_

In [3]:
y = sklearn.preprocessing.OneHotEncoder(sparse=False).fit_transform(y.reshape(-1, 1))
X = sklearn.preprocessing.MinMaxScaler((0, 1)).fit_transform(X)
print(f'X shape: {X.shape}\nClasses: {np.unique(y)}')
print(f'X max: {X.max()} X min {X.min()}')

X shape: (11788, 108)
Classes: [0. 1.]
X max: 1.0 X min 0.0


In [4]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)
print(f'X shape: {X.shape}\nY shape: {y.shape}')
print(f'X_test shape: {X_test.shape}\nY_test shape: {y_test.shape}')

X shape: (10609, 108)
Y shape: (10609, 200)
X_test shape: (1179, 108)
Y_test shape: (1179, 200)


In [5]:
x_train = torch.tensor(X, dtype=torch.float)
print(x_train.shape)

torch.Size([10609, 108])


In [6]:
y_train = torch.zeros((y.shape[0], y.shape[1]), dtype=torch.float)
y_train = torch.tensor(y, dtype=torch.float)
x_test = x_train
n_classes = y_train.size(1)
print(n_classes)
print(y_train)
y_train.sum(dim=0)

# torch.cuda.set_device(0)
n_classes = y_train.shape[1]
device = torch.device("cpu")
x_train = x_train.to(device)
y_train = y_train.argmax(dim=1).to(torch.long).to(device)
loss_form = torch.nn.NLLLoss() # CrossEntropyLoss()
y_train

200
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


tensor([ 66,   9, 143,  ..., 167, 183,  47])

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
dimensions = [200, 200]
layers = [
    torch.nn.Linear(x_train.size(1), dimensions[0] * n_classes),
    torch.nn.LeakyReLU(),
    lens.nn.XLinear(dimensions[0], dimensions[1], n_classes),
    torch.nn.Softmax(),
]
model = torch.nn.Sequential(*layers)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
model.train()
need_pruning = True
for epoch in range(3000):
    print("Epoch:", epoch)
    # forward pass
    optimizer.zero_grad()
    y_pred = model(x_train).squeeze()
    y_train = y_train.to(torch.long)
    # print("x_train", x_train.shape, x_train.dtype)
    # print("y_pred", y_pred.shape, y_pred.dtype)
    # print("y_train", y_train.shape, y_train.dtype)
    # Compute Loss
    loss = loss_form(torch.log(y_pred), y_train)

    for module in model.children():
        if isinstance(module, torch.nn.Linear):
            loss += 1e-8 * torch.norm(module.weight, 1)
            break

    # backward pass
    loss.backward()
    optimizer.step()

    if epoch > 1500 and need_pruning:
        prune_features(model, n_classes, device=device)
        need_pruning = False

    # compute accuracy
    if (epoch + 1) % 10 == 0:
        y_pred_d = torch.argmax(y_pred, dim=1)
        y_train_d = y_train # torch.argmax(y_train, dim=1)
        accuracy = y_pred_d.eq(y_train_d).sum().item() / y_train.size(0) * 100.
        print(f'Epoch: {epoch + 1} train accuracy: {accuracy:.2f} loss: {loss:.4f}')

Epoch: 0


# Local explanations

In [ ]:

np.set_printoptions(precision=2, suppress=True)
outputs = []
for i, (xin, yin) in enumerate(zip(x_train, y_train)):
    model_reduced = get_reduced_model(model, xin)
    for module in model_reduced.children():
        if isinstance(module, torch.nn.Linear):
            wa = module.weight.detach().numpy()
            ba = module.bias.detach().numpy()
            break
    output = model_reduced(xin)

    pred_class = torch.argmax(output)
    true_class = torch.argmax(y_train[i])

    # generate local explanation only if the prediction is correct
    if pred_class.eq(true_class):
        local_explanation = logic.relu_nn.explain_local(model, x_train, y_train, xin)
        print(f'Input {(i + 1)}')
        print(f'\tx={xin.detach().numpy()}')
        print(f'\ty={output.detach().numpy()}, y_label={yin}')
        print(f'\tw={wa}')
        print(f'\tb={ba}')
        print(f'\tExplanation: {local_explanation}')
        print()
        xin = xin.reshape(8, 8)
        plt.figure(1, figsize=(3, 3))
        plt.imshow(xin, cmap=plt.cm.gray_r, interpolation='nearest')
        plt.show()
#         wa = wa.reshape(8, 8)
#         plt.figure(1, figsize=(3, 3))
#         plt.imshow(wa * xin.numpy(), cmap=plt.cm.gray_r, interpolation='nearest')
#         plt.show()

    outputs.append(output)
    if i > 10:
        break

In [ ]:
# %% md

# Combine local explanations

# %%
counters = []
from sklearn.metrics import f1_score
y_train_d = torch.argmax(y_train, dim=1)
for target_class in range(n_classes):
    global_explanation, predictions, counter = logic.combine_local_explanations(model, x_train, y_train,
                                                                              topk_explanations=10,
                                                                              target_class=target_class,
                                                                              concept_names=concept_names)

    y2 = torch.argmax(y_train, dim=1) == target_class
    accuracy = sum(predictions == y2.detach().numpy().squeeze()) / len(predictions)
    f1 = f1_score(y_train[:, target_class], predictions)
    print(f'Class {target_class} - Global explanation: "{global_explanation}" - Accuracy: {accuracy:.4f} - F1: {f1:.4f}')
    counters.append(counter)


In [ ]:
for i, counter in enumerate(counters):
    for j, values in enumerate(counter.items()):
        print(i, j, values)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
y_pred = model(torch.Tensor(X_test)).argmax(dim=1).detach().numpy()
y_test = np.argmax(y_test, axis=1)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"Accuracy: {accuracy:.2f}.\nF1: {f1}")

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier(max_depth=30)
X_bool = X > 0.5
tree_model.fit(X, y_train_d.cpu().numpy())
X_test_bool = X_test > 0.5

y_pred = tree_model.predict(X)
accuracy = accuracy_score(y_train.cpu().numpy(), y_pred)
f1 = f1_score(y_train.cpu().numpy(), y_pred, average='macro')
print(f"Accuracy: {accuracy:.2f}.\nF1: {f1:.2f}")

y_pred = tree_model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='macro')
print(f"Accuracy: {accuracy:.2f}.\nF1: {f1:.2f}")

In [ ]:
sklearn.tree.plot_tree(tree_model)
plt.show()